In [ ]:
#This is used to see the GPU configuration
##In our case we are using googe colab freebie version that is Tesla T4

!nvidia-smi

Sun Nov 26 12:16:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import Sequential
from keras.layers import Dense,Embedding,Flatten, LSTM

In [ ]:
(X_train,y_train),(X_test,y_test) = imdb.load_data()

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
X_train.shape

(25000,)

In [ ]:
X_train = pad_sequences(X_train,padding='post',maxlen=50)
X_test = pad_sequences(X_test,padding='post',maxlen=50)

In [ ]:
X_train.shape

(25000, 50)

In [ ]:
model = Sequential()

##Adding embedding layer to convert into vector representation
##Here 10000 represents intructs the model to consider top 10000 unique words and
##convert them into their vector respresentation. Please note in train we have 25000 words out of which
##We are going to utilize the top 10000 frequent words and there occurrence we are going to observe in the each
##Sentence.
##output_dim represents the dimensions of output vector resprsentation in space. 1 represents representing in 1
##2 represents 2 D .. so on and input_length  is equivalent to max lenght that we have considered above for
##while applying padding.
##Also, please note that size of Embedding layer is a hyperparameter and may varry as per experimentation
model.add(Embedding(10000,output_dim=1,input_length=50))

##Adding LSTM layers where 32 represents the number of hidden neuron
##See we are variatins of LSTM like one to many, many to many etc. Here 
##we are solving the sentiment analysis use case where we do not require any sequence in return.
##This is the reason why we are setting return_sequence to False. Ideally LSTM takes a sequence and returns the sequece
##Since these all are sequece to sequence models. When we just need output and not the return sequence then set
##return_sequences=False
model.add(LSTM(32,return_sequences=False))

##Output layer where we are having only 1 neuron since this is a Classification problem statement
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 1)             10000     
                                                                 
 lstm (LSTM)                 (None, 32)                4352      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 14385 (56.19 KB)
Trainable params: 14385 (56.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train,epochs=1,validation_data=(X_test,y_test))

##In order to save the time we are only training the model for 1 epochs

# Important observation:
When previous lecture we trained the imdb dataset on top of RNN model which was trained over >1 epocs and was still giving around 50% accuracy score. 

But, here in case of LSTM when trained over same dataset with just 1 epoch we are getting accuracy of around 80%.

This show how good LSTM is over RNN

In [ ]:
##Now we will be doing the prediction and for that we cannot straight away pass the test datapoints
##to the model since dimension of datapoint and the dimension in which model understands the datapoints are
##different

##Dimension of datapoint before reshape

X_test[0].shape

(50,)

In [ ]:
##Dimension of the datapoint after reshape

test_data = X_test[0][0:50].reshape(1,-1)

In [ ]:
test_data.shape

(1, 50)

# Please note that our purpose of reshaping here is to covert the input in terms of columns(1,50) which intially is present in terms of rows(50,)

In [ ]:
model.predict(test_data)

##Here we are getting output as 0.24039903 which is less than 0.5
##hence it will be squashed to 0 which is negative sentiment (recall sigmoid actication fun^ basics)

1/1 [==============================] - 1s 573ms/step


array([[0.24039903]], dtype=float32)

# Logic to squash predicted output to 0 or 1:
Here we can write a logic where if predicted_output >=0.5 then it will be treated as 1(positive sentiment) else it will be treated as 0(negative sentiment)

# Please note that we are using inbult dataset called imdb dataset which is already preprocessed. Already preprocessed does not means that it is converted into vector representation. We still need to explicit to do it. Here pre processed dataset means removing noice in terms of stemming, lowercase conversion, lemmitization, html tag removal, special symbol removal etc. Refer to Lec-7 folder where we have a .ipynb implementation of the same.

# In real word scenario we will be getting the dataset which was not already preprocessed and will require to perform preprocessing on top of that

# Assigment
Dataset: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

So this is unprocessed imdb dataset where you will find html tags, special symbols etc in the **Review** column. 
We need to preprocess this first. Then if you observe the output label **Sentiment** then it is categorical encoded i.e; positive and negative. We need to perform integer encoding on this where positive will be represented by 1 and negative by 0.

Once this is done we may use the name LSTM implementation being discussed here for builting and training the model